In [9]:
import sys
from sqlalchemy import create_engine
import pymysql
import pandas as pd

In [10]:
import random
random.seed(1030)

In [53]:
import numpy as np
import datetime
from datetime import timedelta, date

### Paths and Connection Details

In [12]:
exec(open("/Users/walkerag/Documents/osu/cs467/project_paths.py").read())

In [13]:
db_connection = create_engine(db_connection_str)

### Tables

In [14]:
all_tables = pd.read_sql('SHOW TABLES', con=db_connection)
print(all_tables)

# Describe all the DB tables
def describe_table(table, db_connection):
        
    print(table)    
    table_cols = pd.read_sql('DESCRIBE ' + table, con=db_connection)
    print(table_cols)

result = [describe_table(table, db_connection) for table in all_tables['Tables_in_project']]

           Tables_in_project
0                    Country
1                       Item
2             Price_Feedback
3             Price_Response
4              Shopping_List
5      Shopping_List_History
6                      State
7                      Store
8             Store_Feedback
9    Store_Feedback_Category
10            Store_Response
11                      User
12           User_Reputation
13  User_Reputation_Category
Country
          Field              Type Null  Key Default           Extra
0    country_id  int(10) unsigned   NO  PRI    None  auto_increment
1  country_name       varchar(50)   NO         None                
Item
              Field              Type Null  Key Default           Extra
0           item_id  int(10) unsigned   NO  PRI    None  auto_increment
1         item_name       varchar(50)   NO         None                
2          item_upc       varchar(12)   NO         None                
3  item_description      varchar(200)   NO         None     

### Load Data

In [44]:
# Function to load data from a CSV into MySQL database
def data_loader_csv(data_path, table_name, db_connection, load_type):
    
    print("Loading data from CSV")
    
    # Read in from CSV
    df = pd.read_csv(data_path + 'Input Data - ' + table_name.lower() + '.csv')
    
    print(df.head(5))
    
    # Load to mySQL
    df.to_sql(table_name, con = db_connection, if_exists = load_type, index = False)
    
    # Check table
    results_df = pd.read_sql('SELECT * FROM ' + table_name + ' LIMIT 5', con=db_connection)
    print(results_df)

In [126]:
data_loader_csv(data_path, "Country", db_connection, "append")

Loading data from CSV
     country_name
0     Afghanistan
1         Albania
2         Algeria
3  American Samoa
4         Andorra
   country_id    country_name
0           8     Afghanistan
1           9         Albania
2          10         Algeria
3          11  American Samoa
4          12         Andorra


In [115]:
data_loader_csv(data_path, "State", db_connection, "append")

Loading data from CSV
  state_name
0         AL
1         AK
2         AZ
3         AR
4         CA
   state_id state_name
0         1         AL
1         2         AK
2         3         AZ
3         4         AR
4         5         CA


In [ ]:
data_loader_csv(data_path, "Item", db_connection, "append")

In [128]:
data_loader_csv(data_path, "User", db_connection, "append")

Loading data from CSV
  first_name last_name                              email signup_date  \
0       Adam    Walker       adamwalker@notarealemail.com  2020-04-20   
1      James      Dean        jamesdean@notarealemail.com  2020-04-20   
2    Zackary     Morse     zackarymorse@notarealemail.com  2020-04-20   
3  Heriberto    Mellor  heribertomellor@notarealemail.com  2020-04-20   
4      Elton    Bussey      eltonbussey@notarealemail.com  2020-04-20   

   last_login  user_country  
0  2020-04-20           235  
1  2020-04-20           235  
2  2020-04-20           235  
3  2020-04-20           235  
4  2020-04-20           235  
   user_id first_name last_name                              email  \
0        1       Adam    Walker       adamwalker@notarealemail.com   
1        2      James      Dean        jamesdean@notarealemail.com   
2        3    Zackary     Morse     zackarymorse@notarealemail.com   
3        4  Heriberto    Mellor  heribertomellor@notarealemail.com   
4        

In [139]:
data_loader_csv(data_path, "Store", db_connection, "append")

Loading data from CSV
     store_name  store_lat  store_long  store_active_flag  \
0  Trader Joe's          0           0                  1   
1  Trader Joe's          0           0                  1   
2  Trader Joe's          0           0                  1   
3  Trader Joe's          0           0                  1   
4  Trader Joe's          0           0                  1   

          store_street           store_city  store_state  store_zip  
0  417 Westlake Center            Daly City            5      94015  
1   301 McLellan Drive  South San Francisco            5      94080  
2       265 Winston Dr        San Francisco            5      94132  
3           401 Bay St        San Francisco            5      94133  
4        3 Masonic Ave        San Francisco            5      94118  
   store_id    store_name  store_lat  store_long  store_active_flag  \
0        67  Trader Joe's        0.0         0.0                  1   
1        68  Trader Joe's        0.0         0.0 

In [45]:
data_loader_csv(data_path, "Store_Feedback_Category", db_connection, "append")

Loading data from CSV
  store_feedback_category
0           Out of Stocks
1                 Pricing
2          Rate Your Trip
3                Cashiers
4                 Hygiene
   store_feedback_category_id store_feedback_category
0                           1           Out of Stocks
1                           2                 Pricing
2                           3          Rate Your Trip
3                           4                Cashiers
4                           5                 Hygiene


In [3]:
cnx = mysql.connector.connect(user='root', password='cs467project',
                              host='localhost',
                              database='romanesco',
                              use_pure=False)

In [ ]:
df = pd.read_sql('DELETE FROM Store', con=db_connection)
print(df)

In [48]:
df = pd.read_sql('DESCRIBE Country', con=db_connection)
print(df)

          Field              Type Null  Key Default           Extra
0    country_id  int(10) unsigned   NO  PRI    None  auto_increment
1  country_name       varchar(50)   NO         None                


In [ ]:
df = pd.read_sql('SELECT * FROM Country', con=db_connection)
print(df)

### Sklearn

In [11]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

In [12]:
X, y = make_hastie_10_2(random_state=0)

In [15]:
X[1:10,1:10]

array([[ 1.45427351,  0.76103773,  0.12167502,  0.44386323,  0.33367433,
         1.49407907, -0.20515826,  0.3130677 , -0.85409574],
       [ 0.6536186 ,  0.8644362 , -0.74216502,  2.26975462, -1.45436567,
         0.04575852, -0.18718385,  1.53277921,  1.46935877],
       [ 0.37816252, -0.88778575, -1.98079647, -0.34791215,  0.15634897,
         1.23029068,  1.20237985, -0.38732682, -0.30230275],
       [-1.42001794, -1.70627019,  1.9507754 , -0.50965218, -0.4380743 ,
        -1.25279536,  0.77749036, -1.61389785, -0.21274028],
       [ 0.3869025 , -0.51080514, -1.18063218, -0.02818223,  0.42833187,
         0.06651722,  0.3024719 , -0.63432209, -0.36274117],
       [-0.35955316, -0.81314628, -1.7262826 ,  0.17742614, -0.40178094,
        -1.63019835,  0.46278226, -0.90729836,  0.0519454 ],
       [ 0.12898291,  1.13940068, -1.23482582,  0.40234164, -0.68481009,
        -0.87079715, -0.57884966, -0.31155253,  0.05616534],
       [ 0.90082649,  0.46566244, -1.53624369,  1.48825219,  1

In [16]:
X_train, X_test = X[:2000], X[2000:]

In [17]:
y_train, y_test = y[:2000], y[2000:]

In [18]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(X_train, y_train)

In [19]:
clf.score(X_test, y_test)

0.913

### Data Simulation

In [140]:
# Function to create and upload a shopping list for a given user_id
# min_items and max_items limit number of items in list
def _fill_list(user_id, min_items, max_items, items):
    
    # Determine total items in list
    tot_items = np.random.uniform(min_items, max_items, 1).round().astype(int)
    print(tot_items)

    # Get item IDs
    list_items = np.random.choice(items['item_id'], tot_items, replace=False)

    # Convert to pandas dataframe
    dataset = pd.DataFrame({'item_id': list_items})

    # Format dataframe
    dataset['user_id'] = user_id
    dataset['time_added'] = datetime.datetime.now()
    dataset['time_removed'] = None
    dataset = dataset[['user_id', 'item_id','time_added','time_removed']]

    # Load to mySQL
    print("Loading")
    dataset.to_sql('Shopping_List_History', con = db_connection, if_exists = "append", index = False)

# Generate shopping list by sampling from items table
def generate_shopping_list_history(db_connection, min_items, max_items):
    
    # Get items
    items = pd.read_sql('SELECT DISTINCT item_id FROM Item', con=db_connection)
    
    # Users
    users = pd.read_sql('SELECT DISTINCT user_id FROM User', con=db_connection)
    
    # Loop over users
    result = [_fill_list(user_id, min_items, max_items, items) for user_id in users['user_id']]

generate_shopping_list_history(db_connection, 1, 40)

In [8]:
# Check result
df = pd.read_sql('SELECT * FROM Shopping_List_History LIMIT 100', con=db_connection)
print(df)

    shopping_list_history_id  user_id  item_id          time_added  \
0                          7        1       52 2020-04-22 06:25:21   
1                          8        1       10 2020-04-22 06:25:21   
2                          9        1       27 2020-04-22 06:25:21   
3                         10        1       42 2020-04-22 06:25:21   
4                         11        1       33 2020-04-22 06:25:21   
5                         12        1       15 2020-04-22 06:25:21   
6                         13        1       31 2020-04-22 06:25:21   
7                         14        1       17 2020-04-22 06:25:21   
8                         15        1       43 2020-04-22 06:25:21   
9                         16        1       22 2020-04-22 06:25:21   
10                        17        1        7 2020-04-22 06:25:21   
11                        18        1       14 2020-04-22 06:25:21   
12                        19        1       35 2020-04-22 06:25:21   
13                  

In [166]:
# Simulate a trip

# Assign a user to a preferred store

# May need to shuffle shopping list

# Accuracy of feedback

# Need to simulate over time

# Most feedback will be bad
# Pick a feedback category
# Assign upvote or downvote
# Assign a likelihood of giving feedback?

# Store quality, by sections?
# Store experience vs store selection vs prices

# Daily promotion dynamics

# Finish with Rate Your Trip
# Need to update store feedback categories


# Need an update shopping list function
# Needs to be able to load shopping list data


In [113]:
# ASSIGN STORE QUALITY

# Gives each store a quality, randomly distributed around a chain average
# Overall quality, price level, out of stock level
def assign_store_quality(db_connection):
    
    print("Assigning store quality")

    # Get distinct chains
    chains = pd.read_sql('SELECT DISTINCT store_name FROM Store', con=db_connection)

    # Assign overall chain quality
    # Higher is better
    chains['chain_quality'] = 0.5
    chains.loc[chains['store_name'] == "Trader Joe's", 'chain_quality'] = 0.7
    chains.loc[chains['store_name'] == "Safeway", 'chain_quality'] = 0.25
    chains.loc[chains['store_name'] == "Whole Foods Market", 'chain_quality'] = 0.8
    chains.loc[chains['store_name'] == "Costco", 'chain_quality'] = 0.55

    # Assign chain price level
    # Higher means more expensive
    chains['chain_price_level'] = 0.5
    chains.loc[chains['store_name'] == "Trader Joe's", 'chain_price_level'] = 0.6
    chains.loc[chains['store_name'] == "Safeway", 'chain_price_level'] = 0.3
    chains.loc[chains['store_name'] == "Whole Foods Market", 'chain_price_level'] = 0.8
    chains.loc[chains['store_name'] == "Costco", 'chain_price_level'] = 0.3

    # Assign out of stock level
    # Higher means more likely to have in-stock
    chains['chain_stock_level'] = 0.5
    chains.loc[chains['store_name'] == "Trader Joe's", 'chain_stock_level'] = 0.4
    chains.loc[chains['store_name'] == "Safeway", 'chain_stock_level'] = 0.6
    chains.loc[chains['store_name'] == "Whole Foods Market", 'chain_stock_level'] = 0.4
    chains.loc[chains['store_name'] == "Costco", 'chain_stock_level'] = 0.8
    
    print(chains.head())
    
    # Get individual stores
    stores = pd.read_sql('SELECT DISTINCT store_name, store_id FROM Store', con=db_connection)

    # Join on
    stores = stores.join(chains.set_index('store_name'), on='store_name')
    
    # Randomly move individual store attributes around overalll chain ones
    stores['store_quality'] = stores['chain_quality'] + np.random.normal(0, 0.05, stores.shape[0])
    stores['store_price_level'] = stores['chain_price_level'] + np.random.normal(0, 0.05, stores.shape[0])
    stores['store_stock_level'] = stores['chain_stock_level'] + np.random.normal(0, 0.05, stores.shape[0])
    
    print(stores.head())
    
    return(stores)

stores = assign_store_quality(db_connection)

Assigning store quality
           store_name  chain_quality  chain_price_level  chain_stock_level
0        Trader Joe's           0.70                0.6                0.4
1             Safeway           0.25                0.3                0.6
2  Whole Foods Market           0.80                0.8                0.4
3              Costco           0.55                0.3                0.8
     store_name  store_id  chain_quality  chain_price_level  \
0  Trader Joe's        67            0.7                0.6   
1  Trader Joe's        68            0.7                0.6   
2  Trader Joe's        69            0.7                0.6   
3  Trader Joe's        70            0.7                0.6   
4  Trader Joe's        71            0.7                0.6   

   chain_stock_level  store_quality  store_price_level  store_stock_level  
0                0.4       0.725095           0.676175           0.323464  
1                0.4       0.785057           0.618761           0.415

In [ ]:
def assign_item_prices(db_connection):
    
    print("Assigning store quality")

    # Get distinct chains
    chains = pd.read_sql('SELECT DISTINCT store_name FROM Store', con=db_connection)

    # Assign overall chain quality
    # Higher is better
    chains['chain_quality'] = 0.5
    chains.loc[chains['store_name'] == "Trader Joe's", 'chain_quality'] = 0.7
    chains.loc[chains['store_name'] == "Safeway", 'chain_quality'] = 0.25
    chains.loc[chains['store_name'] == "Whole Foods Market", 'chain_quality'] = 0.8
    chains.loc[chains['store_name'] == "Costco", 'chain_quality'] = 0.55

    # Assign chain price level
    # Higher means more expensive
    chains['chain_price_level'] = 0.5
    chains.loc[chains['store_name'] == "Trader Joe's", 'chain_price_level'] = 0.6
    chains.loc[chains['store_name'] == "Safeway", 'chain_price_level'] = 0.3
    chains.loc[chains['store_name'] == "Whole Foods Market", 'chain_price_level'] = 0.8
    chains.loc[chains['store_name'] == "Costco", 'chain_price_level'] = 0.3

    # Assign out of stock level
    # Higher means more likely to have in-stock
    chains['chain_stock_level'] = 0.5
    chains.loc[chains['store_name'] == "Trader Joe's", 'chain_stock_level'] = 0.4
    chains.loc[chains['store_name'] == "Safeway", 'chain_stock_level'] = 0.6
    chains.loc[chains['store_name'] == "Whole Foods Market", 'chain_stock_level'] = 0.4
    chains.loc[chains['store_name'] == "Costco", 'chain_stock_level'] = 0.8
    
    print(chains.head())
    
    # Get individual stores
    stores = pd.read_sql('SELECT DISTINCT store_name, store_id FROM Store', con=db_connection)

    # Join on
    stores = stores.join(chains.set_index('store_name'), on='store_name')
    
    # Randomly move individual store attributes around overalll chain ones
    stores['store_quality'] = stores['chain_quality'] + np.random.normal(0, 0.05, stores.shape[0])
    stores['store_price_level'] = stores['chain_price_level'] + np.random.normal(0, 0.05, stores.shape[0])
    stores['store_stock_level'] = stores['chain_stock_level'] + np.random.normal(0, 0.05, stores.shape[0])
    
    print(stores.head())
    
    return(stores)

stores = assign_store_quality(db_connection)

In [46]:
# Get store feedback categories
store_feedback = pd.read_sql('SELECT * FROM Store_Feedback_Category', con=db_connection)

In [47]:
print(store_feedback)

    store_feedback_category_id store_feedback_category
0                            1           Out of Stocks
1                            2                 Pricing
2                            3          Rate Your Trip
3                            4                Cashiers
4                            5                 Hygiene
5                            6                 Parking
6                            7                 Produce
7                            8               Selection
8                            9                Checkout
9                           10                Pharmacy
10                          11             Gas Station
11                          12                  Bakery
12                          13                    Deli
13                          14                 General
14                          15                 Returns


In [50]:
# Get users
users = pd.read_sql('SELECT DISTINCT user_id FROM User', con=db_connection)
print(users.head())

def assign_preferred_store(users, stores):

    print("Assigning a preferred store to users")
    users['store_id'] = np.random.choice(stores['store_id'], users.shape[0], replace=True)
    
    return(users)
    
users = assign_preferred_store(users, stores)
print(users.head())

   user_id
0        1
1        2
2        3
3        4
4        5
Assigning a preferred store to users
   user_id  store_id
0        1        77
1        2        74
2        3        83
3        4        86
4        5        80


In [81]:
start_date = datetime.datetime(2020, 1, 1)
print(start_date + timedelta(5))

2020-01-06 00:00:00


In [91]:
# start_date = '2020-01-01'
start_date = datetime.datetime(2020, 1, 1)
users['next_trip_date'] = datetime.datetime(2020, 1, 1)
day_count = 60

def take_a_trip(sim_date, u):
    
    print("Take a trip")
    print(u)

for sim_date in (start_date + timedelta(n) for n in range(day_count)):
    
    # Simulate a day
    print(sim_date)
    
    # Delete shopping list data
    # Pull new shopping list using historical data
    
    # Loop over users
    for u in users['user_id'][1:5]:
        
        # Get date of user's next grocery trip
        print(u)        
        user_row = users[users['user_id'] == u]        
        next_trip_date = user_row.iloc[0]['next_trip_date']
        print(next_trip_date)
        
        # If today, have the user take a grocery trip
        if next_trip_date == sim_date:
            
            print("Taking a grocery trip!")
            
            # Update shopping list
            
            # Go to the store
            
            # Critique feedback            
            
            # Update date of next trip
            # Randomly choose a value
            days_until_next_trip = int(np.random.poisson(7, 1))
            users.loc[users['user_id'] == u, "next_trip_date"] = sim_date + timedelta(days_until_next_trip)
                        

2020-01-01 00:00:00
2
2020-01-01 00:00:00
Taking a grocery trip!
3
2020-01-01 00:00:00
Taking a grocery trip!
4
2020-01-01 00:00:00
Taking a grocery trip!
5
2020-01-01 00:00:00
Taking a grocery trip!
2020-01-02 00:00:00
2
2020-01-12 00:00:00
3
2020-01-06 00:00:00
4
2020-01-14 00:00:00
5
2020-01-07 00:00:00
2020-01-03 00:00:00
2
2020-01-12 00:00:00
3
2020-01-06 00:00:00
4
2020-01-14 00:00:00
5
2020-01-07 00:00:00
2020-01-04 00:00:00
2
2020-01-12 00:00:00
3
2020-01-06 00:00:00
4
2020-01-14 00:00:00
5
2020-01-07 00:00:00
2020-01-05 00:00:00
2
2020-01-12 00:00:00
3
2020-01-06 00:00:00
4
2020-01-14 00:00:00
5
2020-01-07 00:00:00
2020-01-06 00:00:00
2
2020-01-12 00:00:00
3
2020-01-06 00:00:00
Taking a grocery trip!
4
2020-01-14 00:00:00
5
2020-01-07 00:00:00
2020-01-07 00:00:00
2
2020-01-12 00:00:00
3
2020-01-17 00:00:00
4
2020-01-14 00:00:00
5
2020-01-07 00:00:00
Taking a grocery trip!
2020-01-08 00:00:00
2
2020-01-12 00:00:00
3
2020-01-17 00:00:00
4
2020-01-14 00:00:00
5
2020-01-12 00:00:0

In [90]:
users['user_id'][1:10]

1     2
2     3
3     4
4     5
5     6
6     7
7     8
8     9
9    10
Name: user_id, dtype: int64

In [85]:
np.random.poisson(7, 100)

array([10, 12,  4,  4,  8,  7, 10,  5, 10,  8,  5, 12,  9, 11,  5,  5, 11,
        4,  5,  4,  5, 10,  4, 13,  2,  6, 12,  4,  7,  9,  4,  9,  7,  4,
       11,  9,  6, 10,  3,  6, 11,  5, 11,  8,  9,  9,  5,  3,  9,  9,  9,
        7, 10,  3,  4, 17,  6, 11,  6,  4,  5,  3,  4,  8,  7,  8,  2, 10,
        7, 10, 10,  6, 10,  7,  7,  6,  7,  4,  6,  4,  2,  6,  7,  5,  9,
        5, 11, 10,  2,  9, 11,  1, 11,  8, 10,  4,  1,  8,  9,  9])

In [51]:
# Simulate a trip schedule

users.head()

users['last_trip_date']





,user_id,store_id
0,1,77
1,2,74
2,3,83
3,4,86
4,5,80


In [148]:
items = pd.read_sql('SELECT DISTINCT item_id FROM Item', con=db_connection)

In [149]:
users = pd.read_sql('SELECT DISTINCT user_id FROM User', con=db_connection)

In [ ]:
def describe_table(table, db_connection):
        
    print(table)    
    table_cols = pd.read_sql('DESCRIBE ' + table, con=db_connection)
    print(table_cols)

result = [fill_list(user_id, min_items, max_items) for user_id in users['user_id']]

In [ ]:
pd.read_sql('''INSERT INTO Shopping_List (user_id, item_id, time_added, shopping_list_history_id, item_quantity)
                  SELECT 
                  user_id
                  , item_id
                  , time_added
                  , shopping_list_history_id
                  , 1 as item_quantity
                  FROM Shopping_List_History
                  WHERE time_removed IS NULL'''
                 , con=db_connection)

In [96]:
df = pd.read_sql('SELECT * FROM Shopping_List', con=db_connection)
print(df)

      shopping_list_id  user_id  item_id          time_added  \
0                    1        1       52 2020-04-22 06:25:21   
1                    2        1       10 2020-04-22 06:25:21   
2                    3        1       27 2020-04-22 06:25:21   
3                    4        1       42 2020-04-22 06:25:21   
4                    5        1       33 2020-04-22 06:25:21   
5                    6        1       15 2020-04-22 06:25:21   
6                    7        1       31 2020-04-22 06:25:21   
7                    8        1       17 2020-04-22 06:25:21   
8                    9        1       43 2020-04-22 06:25:21   
9                   10        1       22 2020-04-22 06:25:21   
10                  11        1        7 2020-04-22 06:25:21   
11                  12        1       14 2020-04-22 06:25:21   
12                  13        1       35 2020-04-22 06:25:21   
13                  14        1        5 2020-04-22 06:25:21   
14                  15        1        3

In [109]:
with db_connection.connect() as connection:
    result = connection.execute('''DELETE FROM Shopping_List''')

In [111]:
with db_connection.connect() as connection:
    result = connection.execute('''
        INSERT INTO Shopping_List (user_id, item_id, time_added, shopping_list_history_id, item_quantity)
        SELECT
        user_id
        , item_id
        , time_added
        , shopping_list_history_id
        , 1 as item_quantity
        FROM Shopping_List_History
        WHERE time_removed IS NULL''')

In [ ]:
for row in result:
    print(row)

In [112]:
df = pd.read_sql('SELECT * FROM Shopping_List', con=db_connection)
print(df)

      shopping_list_id  user_id  item_id          time_added  \
0                 8192        1       52 2020-04-22 06:25:21   
1                 8193        1       10 2020-04-22 06:25:21   
2                 8194        1       27 2020-04-22 06:25:21   
3                 8195        1       42 2020-04-22 06:25:21   
4                 8196        1       33 2020-04-22 06:25:21   
5                 8197        1       15 2020-04-22 06:25:21   
6                 8198        1       31 2020-04-22 06:25:21   
7                 8199        1       17 2020-04-22 06:25:21   
8                 8200        1       43 2020-04-22 06:25:21   
9                 8201        1       22 2020-04-22 06:25:21   
10                8202        1        7 2020-04-22 06:25:21   
11                8203        1       14 2020-04-22 06:25:21   
12                8204        1       35 2020-04-22 06:25:21   
13                8205        1        5 2020-04-22 06:25:21   
14                8206        1        3

In [ ]:
df = pd.read_sql('DELETE FROM Shopping_List_History', con=db_connection)
print(df)

[15]
Loading
[30]
Loading
[39]
Loading
[12]
Loading
[35]
Loading
[30]
Loading
[28]
Loading
[33]
Loading
[8]
Loading
[6]
Loading
[1]
Loading
[32]
Loading
[26]
Loading
[10]
Loading
[2]
Loading
[25]
Loading
[20]
Loading
[39]
Loading
[7]
Loading
[17]
Loading
[17]
Loading
[13]
Loading
[31]
Loading
[13]
Loading
[10]
Loading
[12]
Loading
[25]
Loading
[24]
Loading
[32]
Loading
[7]
Loading
[25]
Loading
[5]
Loading
[16]
Loading
[36]
Loading
[18]
Loading
[34]
Loading
[14]
Loading
[33]
Loading
[15]
Loading
[3]
Loading
[17]
Loading
[13]
Loading
[15]
Loading
[16]
Loading
[31]
Loading
[6]
Loading
[3]
Loading
[26]
Loading
[36]
Loading
[23]
Loading
[28]
Loading
[20]
Loading
[20]
Loading
[33]
Loading
[16]
Loading
[25]
Loading
[12]
Loading
[8]
Loading
[32]
Loading
[40]
Loading
[22]
Loading
[30]
Loading
[38]
Loading
[3]
Loading
[34]
Loading
[24]
Loading
[27]
Loading
[10]
Loading
[37]
Loading
[39]
Loading
[39]
Loading
[29]
Loading
[13]
Loading
[34]
Loading
[35]
Loading
[21]
Loading
[9]
Loading
[9]
Loading


### Scrap

In [ ]:
# initialize list of lists 
data = [['United States of America'], ['Brazil'], ['Spain']] 
  
# Create the pandas DataFrame 
df = pd.DataFrame(data, columns = ['country_name']) 
  
# print dataframe. 
df 